# iCAT Export
---

In [3]:
import os
import sys
import re
import subprocess
from multiprocessing import Pool
from pathlib import Path
from random import sample
from functools import partial
from shutil import rmtree

from ruamel.yaml import YAML
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
from skimage.io import imsave

import renderapi
from renderapi.client import ArgumentParameters

from icatapi.render_pandas import *
from icatapi.plotting import *
from icatapi.render_transforms import scale_stack

#### Settings

In [4]:
# `pandas` display settings
# -----------------------
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 15)

# `altair` settings
# ---------------
alt.data_transformers.disable_max_rows();

## Set up `render-ws` environment
---

In [6]:
# `render` project parameters
# ---------------------------
owner = 'rlane'
project = '20210612_UMCU5I'
project_dir = Path(f'/long_term_storage/{owner}/SECOM/projects/{project}')
export_dir = Path(f'/long_term_storage/{owner}/CATMAID/projects/{project}')

# Create a renderapi.connect.Render object
# ----------------------------------------
render_connect_params = {
    'host': 'sonic.tnw.tudelft.nl',
    'port': 8080,
    'owner': owner,
    'project': project,
    'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
    'memGB': '2G'
}
render = renderapi.connect(**render_connect_params)

# Infer stack and section info
# ----------------------------
stacks = renderapi.render.get_stacks_by_owner_project(render=render)
stacks_EM = [stack for stack in stacks if 'EM' in stack]
stacks_FM = [stack for stack in stacks if 'EM' not in stack]
stacks_2_export = [
    'lil_EM_stitched',
    'hoechst_correlated'
]

# Output
# ------
out = f"""\
project directory... {project_dir} | Exists: {project_dir.exists()}
export directory.... {export_dir} | Exists: {export_dir.exists()}
all stacks.......... {stacks}
EM stacks........... {stacks_EM}
FM stacks........... {stacks_FM}
stacks to export.... {stacks_2_export}
...
"""
print(out)

# Create project DataFrame
# ------------------------
df_project = create_stacks_DataFrame(stacks=stacks_2_export,
                                     render=render)
df_project.groupby('stack')\
          .apply(lambda x: x.head(5))

project directory... /long_term_storage/rlane/SECOM/projects/20210612_UMCU5I | Exists: True
export directory.... /long_term_storage/rlane/CATMAID/projects/20210612_UMCU5I | Exists: False
all stacks.......... ['hoechst_correlated', 'big_EM_correlated', 'big_EM_overlaid', 'hoechst_overlaid', 'lil_EM_minimontages', 'lil_EM_world', 'hoechst_world', 'big_EM_world', 'lil_EM_stitched', 'lil_EM_montaged', 'hoechst_montaged', 'lil_EM_filtered', 'lil_EM', 'hoechst', 'big_EM']
EM stacks........... ['big_EM_correlated', 'big_EM_overlaid', 'lil_EM_minimontages', 'lil_EM_world', 'big_EM_world', 'lil_EM_stitched', 'lil_EM_montaged', 'lil_EM_filtered', 'lil_EM', 'big_EM']
FM stacks........... ['hoechst_correlated', 'hoechst_overlaid', 'hoechst_world', 'hoechst_montaged', 'hoechst']
stacks to export.... ['lil_EM_stitched', 'hoechst_correlated']
...



tileId    z   width  height   minint   maxint  \
stack                                                                         
hoechst_corr... 1773  aag_hoechst...  1.0  2048.0  2048.0   7500.0  14000.0   
                1774  aah_hoechst...  1.0  2048.0  2048.0   7500.0  14000.0   
                1775  aai_hoechst...  1.0  2048.0  2048.0   7500.0  14000.0   
                1776  aaj_hoechst...  1.0  2048.0  2048.0   7500.0  14000.0   
                1777  aak_hoechst...  1.0  2048.0  2048.0   7500.0  14000.0   
lil_EM_stitched 0     aaa_lil_EM-...  1.0  4096.0  4096.0  29000.0  34000.0   
                1     aab_lil_EM-...  1.0  4096.0  4096.0  29000.0  34000.0   
                2     aac_lil_EM-...  1.0  4096.0  4096.0  29000.0  34000.0   
                3     aad_lil_EM-...  1.0  4096.0  4096.0  29000.0  34000.0   
                4     aae_lil_EM-...  1.0  4096.0  4096.0  29000.0  34000.0   

                            imageUrl          tforms           stack  \
stack                                                                  
hoechst_corr... 1773  file:///lon...  [M=[[1.0000...  hoechst_cor...   
                1774  file:///lon...  [M=[[1.0000...  hoechst_cor...   
                1775  file:///lon...  [M=[[1.0000...  hoechst_cor...   
                1776  file:///lon...  [M=[[1.0000...  hoechst_cor...   
                1777  file:///lon...  [M=[[1.0000...  hoechst_cor...   
lil_EM_stitched 0     file:///lon...  [M=[[0.9745...  lil_EM_stit...   
                1     file:///lon...  [M=[[0.9744...  lil_EM_stit...   
                2     file:///lon...  [M=[[0.9734...  lil_EM_stit...   
                3     file:///lon...  [M=[[0.9779...  lil_EM_stit...   
                4     file:///lon...  [M=[[0.9793...  lil_EM_stit...   

                     sectionId  imageRow  imageCol       stageX       stageY  \
stack                                                                          
hoechst_corr... 1773      S001         4         5  -549.896074 -4790.577263   
                1774      S001         4         4  -698.931109 -4753.198759   
                1775      S001         4         3  -847.629471 -4715.826625   
                1776      S001         4         2  -996.170035 -4678.490608   
                1777      S001         4         1 -1145.098396 -4641.152246   
lil_EM_stitched 0         S001        44        44  -594.243000 -4878.719000   
                1         S001        44        43  -611.288000 -4874.399000   
                2         S001        44        42  -628.311000 -4869.990000   
                3         S001        44        41  -645.472000 -4865.776000   
                4         S001        44        40  -662.444000 -4861.482000   

                      rotation  
stack                           
hoechst_corr... 1773      14.0  
                1774      14.0  
                1775      14.0  
                1776      14.0  
                1777      14.0  
lil_EM_stitched 0         14.0  
                1         14.0  
                2         14.0  
                3         14.0  
                4         14.0

## Export `render-ws` stacks to CATMAID
---
### Set up CATMAID export parameters

In [7]:
class CatmaidBoxesParameters(ArgumentParameters):
    """Subclass of `ArgumentParameters` for facilitating CATMAID export client script"""
    def __init__(self, stack, root_directory,
                 height=1024, width=1024, fmt='png', max_level=0,
#                  renderGroup=None,
                 host=None, port=None, baseurl=None,
                 owner=None, project=None, render=None, **kwargs):

        super(CatmaidBoxesParameters, self).__init__(**kwargs)

        self.stack = stack
        self.rootDirectory = root_directory
        self.height = height
        self.width = width
        self.format = fmt
        self.maxLevel = max_level

        render_kwargs = render.make_kwargs()
        host = render_kwargs.get('host')
        port = render_kwargs.get('port')
        self.baseDataUrl = renderapi.render.format_baseurl(host, port)
        self.owner = render_kwargs.get('owner') if owner is None else owner
        self.project = render_kwargs.get('project') if project is None else project

#### Logic for maximum zoom level

Ideally `max_level` is set such that

\begin{equation}
\left( \frac{w_s}{w_t \,\, 2^m} \right) < 1
\end{equation}

where $m$ is `max_level`, $w_s$ is the width of the stack and $w_t$ is the width of each tile. Then

\begin{equation}
m = \textrm{ceil} \left( \log{\frac{w_s}{w_t}} \times \frac{1}{\log{2}} \right)
\end{equation}

In [8]:
# Initialize collection for export parameters
export_data = {}

# Iterate through stacks
for stack, df_stack in df_project.groupby('stack'):

    # Determine `max_level` such that the full section is in view when fully zoomed out
    w_tile = 1024
    h_tile = 1024
    stack_bounds = renderapi.stack.get_stack_bounds(stack=stack,
                                                    render=render)
    w_stack = max(stack_bounds['maxX'] - stack_bounds['minX'],
                  stack_bounds['maxY'] - stack_bounds['minY'])
    max_level = int(np.ceil(np.log(w_stack / w_tile) * 1/np.log(2)))

    # Set parameters for export to CATMAID
    export_params = CatmaidBoxesParameters(stack=stack,
                                           root_directory=export_dir.parent.as_posix(),
                                           width=w_tile,
                                           height=h_tile,
                                           max_level=max_level,
                                           fmt='png',
                                           project=project,
                                           render=render)

    # Add CATMAID export parameters to collection
    export_data[stack] = export_params

# Preview
stack = sample(export_data.keys(), 1)[0]
list(export_data[stack].to_java_args())

['--stack',
 'hoechst_correlated',
 '--rootDirectory',
 '/long_term_storage/rlane/CATMAID/projects',
 '--height',
 '1024',
 '--width',
 '1024',
 '--format',
 'png',
 '--maxLevel',
 '8',
 '--baseDataUrl',
 'http://sonic.tnw.tudelft.nl:8080/render-ws/v1',
 '--owner',
 'rlane',
 '--project',
 '20210612_UMCU5I']

### Call render script
`render_catmaid_boxes.sh`
```sh
Usage: java -cp <render-module>-standalone.jar
      org.janelia.render.client.BoxClient [options] Z values for layers to
      render
  Options:
  * --baseDataUrl
      Base web service URL for data (e.g. http://host[:port]/render-ws/v1)
    --binaryMask
      use binary mask (e.g. for DMG data)
      Default: false
    --createIGrid
      create an IGrid file
      Default: false
    --doFilter
      Use ad hoc filter to support alignment
      Default: false
    --filterListName
      Apply this filter list to all rendering (overrides doFilter option)
    --forceGeneration
      Regenerate boxes even if they already exist
      Default: false
    --format
      Format for rendered boxes
      Default: png
  * --height
      Height of each box
    --help
      Display this note
    --label
      Generate single color tile labels instead of actual tile images
      Default: false
    --maxLevel
      Maximum mipmap level to generate
      Default: 0
    --maxOverviewWidthAndHeight
      Max width and height of layer overview image (omit or set to zero to
      disable overview generation)
    --numberOfRenderGroups
      Total number of parallel jobs being used to render this layer (omit if
      only one job is being used)
  * --owner
      Stack owner
  * --project
      Stack project
    --renderGroup
      Index (1-n) that identifies portion of layer to render (omit if only one
      job is being used)
  * --rootDirectory
      Root directory for rendered tiles (e.g.
      /tier2/flyTEM/nobackup/rendered_boxes)
    --skipInterpolation
      skip interpolation (e.g. for DMG data)
      Default: false
  * --stack
      Stack name
  * --width
      Width of each box
```

#### Wrapper for `render_catmaid_boxes` script for multiprocessing
Multiprocessing is done across sections, so a process is created for each section.

In [9]:
def run_render_catmaid_boxes(z, client_script, java_args):
    """Wrapper for `render_catmaid_boxes` script to enable multiprocessing"""
    p = subprocess.run([client_script.as_posix(), f'{z:.0f}'] + java_args)

#### \*\****COMPUTATIONALLY EXPENSIVE*** \**

##### Run `render_catmaid_boxes` on `N_cores`

`renderapi.client.WithPool` ends prematurely (after exporting one section).  
Weirdly only happens with `lil_EM_montaged` stack...

In [ ]:
# Path to `render_catmaid_boxes` shell script
fp_client = Path(render_connect_params['client_scripts']) / 'render_catmaid_boxes.sh'
# Set number of cores for multiprocessing
N_cores = min(30, df_project['z'].unique().size)

# Iterate through stacks to export
# for stack in tqdm(stacks_2_export):
for stack in stacks_2_export:

    # Create java arguments from export parameters
    java_args = list(export_data[stack].to_java_args())

    # Loop through z values
    z_values = renderapi.stack.get_z_values_for_stack(stack, render=render)
    for z in tqdm(z_values):

        # Execute client script
        p = subprocess.run([fp_client.as_posix(), f'{z:.0f}'] + java_args)

#     # TODO: this breaks for `lil_EM_montaged` stack in weird ways
#     #       so fix it...
#     # Set up `render_catmaid_boxes` client script
#     render_catmaid_boxes_partial = partial(run_render_catmaid_boxes,
#                                            client_script=fp_client,
#                                            java_args=java_args)

# #     # Run `render_catmaid_boxes` across `N_cores`
# #     with renderapi.client.WithPool(N_cores) as pool:l
# #         pool.map(render_catmaid_boxes_partial, z_values)

#     # Run `render_catmaid_boxes` across `N_cores`
#     with Pool(N_cores) as pool:
#         pool.map(render_catmaid_boxes_partial, z_values)

  0%|          | 0/1 [00:00<?, ?it/s]

## Set up tiles for import to CATMAID
---
### Resort CATMAID tiles
By (unchangeable) default, `render_catmaid_boxes` exports tiles as

`root directory` / `project` / `stack` / `width x height` / `zoomlevel` / `z` / `row` / `col.ext`

This is ok, but preferred format for importing to CATMAID is [tile source convention 1](https://catmaid.readthedocs.io/en/stable/tile_sources.html#tile-source-types) --- "[File-based image stack](https://catmaid.readthedocs.io/en/stable/tile_sources.html#file-based-image-stack)"

`root directory` / `project` / `stack` / `z` / `row_col_zoomlevel.ext`

#### \*\****CHANGES LOTS & LOTS OF FILEPATHS ON DISK*** \**

In [ ]:
# # Iterate through stacks to export
# for stack in tqdm(stacks_2_export):

#     # Loop through all the exported tiles per stack
#     fps = (export_dir / stack).glob('1024x1024/**/[0-9]*.png')
#     for fp in fps:

#         # Extract tile info from filepath
#         zoom_level = int(fp.parents[2].name)
#         z = int(fp.parents[1].name)
#         row = int(fp.parents[0].name)
#         col = int(fp.stem)
#         ext = fp.suffix

#         # Reformat tile
#         tile_format_1 = export_dir / stack / f"{z}/{row}_{col}_{zoom_level}{ext}"
#         tile_format_1.parent.mkdir(parents=True, exist_ok=True)
#         fp.rename(tile_format_1)

#     # Clean up (now presumably empty) directory tree
#     rmtree((export_dir / stack / '1024x1024').as_posix())

### Make thumbnails

In [ ]:
# # Iterate through stacks to export
# for stack in tqdm(stacks_2_export):

#     # Get stack bounds & min z value
#     bounds = renderapi.stack.get_stack_bounds(stack=stack,
#                                               render=render)
#     z = bounds['minZ']
#     width = bounds['maxX'] - bounds['minX']
#     scale = 128 / width

#     # Render section image
#     thumbnail = renderapi.image.get_section_image(stack=stack,
#                                                   z=z,
#                                                   scale=scale,
#                                                   render=render)

#     # Save thumbnail to min z value directory
#     # TODO: figure out how thumbnail image actually gets read
#     #       from networking tools it appears to try to load
#     #       http://sonic/long_term_storage/rlane/CATMAID/projects/{project}/{stack}/0/small.png
#     #       but one time it tried to read
#     #       http://sonic/long_term_storage/rlane/CATMAID/projects/20191230_RL010/lil_EM_montaged/4/small.png
#     #       so maybe it's best (easiest) to render min z section image and copy paste into all section directories?
#     fp = export_dir / stack / f"{z:.0f}/small.png"
#     imsave(fp, thumbnail)

### Create `project.yaml` file

In [ ]:
# Set project yaml file
project_yaml = export_dir / 'project.yaml'

# Collect stack data
stack_data = []
for stack in stacks_2_export:

    # Get dimension data
    bounds = renderapi.stack.get_stack_bounds(stack=stack,
                                              render=render)
    dimensions = (int((bounds['maxX'] - bounds['minX']) * 1.1),
                  int((bounds['maxY'] - bounds['minY']) * 1.1),
                  int(bounds['maxZ']) + 1)

    # Get resolution data
    stack_metadata = renderapi.stack.get_full_stack_metadata(stack=stack,
                                                         render=render)
    resolution = (np.round(stack_metadata['currentVersion']['stackResolutionX'], 5),
                  np.round(stack_metadata['currentVersion']['stackResolutionY'], 5),
                  np.round(stack_metadata['currentVersion']['stackResolutionZ'], 5))

    # Project data for output to project yaml file
    stack_datum = {
        "title": f"{stack}",
        "dimension": f"{dimensions}",
        "resolution": f"{resolution}",
        "zoomlevels": f"{(max_level + 1):.0f}",
        "mirrors": [{
            "title": "Great title",
            "tile_width": 1024,
            "tile_height": 1024,
            "tile_source_type": 1,
            "fileextension": "png",
            "url": f"http://sonic.tnw.tudelft.nl{(export_dir/stack).as_posix()}"
        }]
    }
    stack_data.append(stack_datum)

# Create dict for input into project yaml file
project_data = {
    "project": {
        "title": f"{project}",
        "stacks": stack_data
    }
}

In [ ]:
out = f"""\
{project_yaml}
--------\
"""
print(out)

yaml = YAML()
yaml.indent(mapping=2, offset=0)
yaml.dump(project_data, project_yaml)
yaml.dump(project_data, sys.stdout)

## Create stacks for predicted data
---

In [ ]:
# from icatapi.importo import *
# from skimage.external import tifffile

In [ ]:
# zs = [1, 3, 4, 7, 9, 10]
# tile_specs = []
# for z in zs:
#     fp = Path(f'/long_term_storage/rlane/SECOM/projects/20200618_RL012/S{z:03d}/insulin_predicted/insulin_predicted-00000x00000.tif')
#     image = tifffile.imread(fp.as_posix())
#     at = renderapi.transform.AffineModel(M00=32, M11=32)

#     ts = renderapi.tilespec.TileSpec(
#         tileId=f'aaa_insulin_predicted-S{z:03d}-00000x00000',
#         z=z,
#         width=image.shape[1],
#         height=image.shape[0],
#         imageUrl=fp.as_uri(),
#         tforms=[at]
#     )
#     tile_specs.append(ts)

# ts.to_dict()

In [ ]:
# stack = 'insulin_predicted'
# renderapi.stack.create_stack(stack,
#                              render=render)

# renderapi.client.import_tilespecs(stack,
#                                   tile_specs,
#                                   render=render)

# renderapi.stack.set_stack_state(stack, 'COMPLETE', render=render)